
<a id='testing'></a>
<div id="qe-notebook-header" style="text-align:right;">
        <a href="https://quantecon.org/" title="quantecon.org">
                <img style="width:250px;display:inline;" src="https://assets.quantecon.org/img/qe-menubar-logo.svg" alt="QuantEcon">
        </a>
</div>

# Pacotes, Testes, e Integração Contínua

## Conteúdo

- [Pacotes, Teste, e Integração Contínua](#Pacotes,-Testes,-e-Integração-Contínua)  
  - [Configuração do Projeto](#Configuração-do-Projeto)  
  - [Estrutura do Projeto](#Estrutura-do-Projeto)  
  - [Fluxo de Trabalho do Projeto](#Fluxo-de-Trabalho-do-Projeto)  
  - [Teste de Unidade](#Teste-de-Unidade)  
  - [Integração Contínua com Travis](#Integração-Contínua-com-Travis)  
  - [Código de Cobertura](#Código-de-Cobertura)  
  - [Pull Requests para Projetos Externos do Julia](#Pull-Requests-para-Projetos-Externos-do-Julia)
  - [Benchmarking](#Benchmarking)  
  - [Notas Adicionais](#Notas-Adicionais)  
  - [Revisão](#Revisão)   
  - [Exercícios](#Exercícios)  
  


> *Devidamente traduzido, revisado e adaptado do [QuantEcon](https://quantecon.org/) pelos bolsistas CNPq, Pedro Luiz H. Furtado e Jonas Aragão M. Corpes, sob supervisão do Prof. Christiano Penna, do CAEN/UFC.*

Um sistema complexo que funciona invariavelmente evoluiu de um sistema simples que funcionou. A proposição inversa também parece ser verdadeira: um sistema complexo projetado do zero nunca funciona e não pode ser feito para funcionar. Você precisa começar de novo, começando com um sistema simples que funciona - [Lei de Gal](https://en.wikipedia.org/wiki/John_Gall_%28author%29#Gall%27s_law).

Em co-autoria com Arnav Sood.

Esta aula discute a estruturação de um projeto como um módulo Julia e o teste com ferramentas do GitHub.

Os benefícios incluem:

- Especificar dependências (e suas versões) para que seu projeto funcione nas configurações de Julia e ao longo do tempo. 
- Ser capaz de carregar as funções do seu projeto de fora sem copiar/colar.  
- Escrever testes executados localmente e *automaticamente no servidor GitHub*.  
- Tendo o GitHub testando seu projeto em sistemas operacionais, versões Julia, etc. 



<a id='project-setup'></a>

## Configuração do Projeto

### Configuração da Conta

#### Travis CI

Como veremos mais tarde, Travis é um serviço que automaticamente testa seu projeto em um server do GitHub.

Primeiro, precisamos ter certeza que sua conta do Github está configurada com Travis CI e Codecov.

Lembre-se de que você se inscreveu no GitHub em [Student Developer Pack](https://education.github.com/pack/) ou [Academic Plan](https://help.github.com/articles/applying-for-an-academic-research-discount/) se elegível.

Navegue até o [site travis-ci.com](https://travis-ci.com/) e clique “sign up with GitHub” – forneça suas credenciais. 

Se tiver dificuldades, veja [tutorial do Travis](https://docs.travis-ci.com/user/tutorial/).

#### Codecov

Codecov é um serviço que informa a abrangência dos seus testes (ou seja, quanto do seu código é realmente testado).

Para se inscrever, visite o [site do Codecov](http://codecov.io/), e clique em “sign up”.

![1](https://github.com/pluiz30/Julia-Hub/assets/60633407/cb8a74da-9caa-4369-9614-9b695f0fbd1c)  


Próximo, clique em “add a repository” e *enable private scope* (isso permite ao Codecov atender seus projetos particulares).

O resultado deve ser:

![2](https://github.com/pluiz30/Julia-Hub/assets/60633407/acda81b1-23cd-4c56-af1d-59da0824da48)  
Isto é tudo que precisamos por enquanto.

### Coniguração do Julia

In [4]:
using InstantiateFromURL
github_project("QuantEcon/quantecon-notebooks-julia", version = "0.5.0", instantiate = true)

Activated C:\Users\cliente\Downloads\Project.toml
Info quantecon-notebooks-julia 0.4.0 activated, 0.5.0 requested


**Nota:** Antes desses passos, verifique se você concluiu a aula de [controle de versão](https://julia.quantecon.org/more_julia/version_control.html) ou executou:

```julia
git config --global user.name "Your Name"
```


**Nota:** Ao longo desta aula, os pontos importantes e as etapas sequenciais do fluxo de trabalho são listados como marcadores.

Para configurar um projeto em Julia:

- Carregue o pacote [PkgTemplates](https://github.com/invenia/PkgTemplates.jl/).

```julia
using PkgTemplates
```


- Crie um *template* para seu projeto: 


Isso especifica os metadados como a licença que usaremos (MIT por padrão), o local (`~/.julia/dev` por padrão), etc.

```julia
ourTemplate = Template(;user="quanteconuser", plugins = [TravisCI(), Codecov()], manifest = true)
```


**Nota**: Certifique-se de substituir o `quanteconuser` com seu ID do GitHub:

- Crie um projeto específico com base neste modelo.

```julia
generate("ExamplePackage.jl", ourTemplate)
```


Se navegarmos para o diretório do pacote, veremos algo como o seguinte.

![3](https://github.com/pluiz30/Julia-Hub/assets/60633407/dac1aaaf-dc14-4373-af8f-a873e9f3fd87)
  
Como lembrete, o local da sua pasta `.julia` pode ser encontrado executando `DEPOT_PATH[1]` em um REPL.

**Nota:** No Mac, isso pode estar oculto; você pode iniciar um terminal `cd ~` e então `cd .julia`, ou tornar [arquivos ocultos visíveis](https://ianlunn.co.uk/articles/quickly-showhide-hidden-files-mac-os-x-mavericks/) no Finder.

### Adicionando um Projeto ao Git

O próximo passo é adicionar este projeto ao controle de versão do Git.

- Abra a tela do repositório em sua conta, conforme discutido anteriormente.


Queremos as seguintes configurações:

![4](https://github.com/pluiz30/Julia-Hub/assets/60633407/42018422-6200-46ca-af09-661e353c890e)  
Em particular:

- O repo que você criar deve ter o mesmo nome que o projeto que adicionamos.
- Devemos deixar as caixas desmarcada para o `README.md`, `LICENSE`, e `.gitignore`, uma vez que estes são tratados pelo `PkgTemplates`.  


Então,

- Arraste e solte sua pasta do seu diretório `~/.julia/dev` para o GitHub Desktop. 
- Clique no botão "publish branch" para enviar seus arquivos para o GitHub.  



Se você navegar para o seu repositório git (o nosso está [aqui](https://github.com/quanteconuser/ExamplePackage.jl/)), você deve ver algo como isso:

![5](https://github.com/pluiz30/Julia-Hub/assets/60633407/efc7dcdd-4fac-4f86-b345-c6829a87fa78)  
**Nota:** Certifique-se de não clonar separadamente o repositório que você adicionou em outro local (por exemplo, na área de trabalho).

Uma observação importante é que você tem algum conjunto de arquivos em sua máquina local (aqui em `~/.julia/dev/ExamplePackage.jl`) e o git está conectado a esses arquivos.

Por conveniência, convém criar um atalho para esse local em algum lugar acessível.

### Adicionando um projeto para o Gerenciador de Pacotes do Julia 

Também queremos que o gerente de pacotes de Julia esteja ciente do projeto.

- Abra um REPL no diretório do projeto recém-criado, observando o caminho expresso acima ou executando o seguinte em um REPL.

```julia
cd(joinpath(DEPOT_PATH[1], "dev", "ExamplePackage"))
```


Observe a falta de `.jl`!

- Execute o seguinte:

```julia
] activate
```


para entrar no ambiente principal de Julia (mais sobre ambientes na segunda metade desta aula).

- e execute:

```julia
] dev .
```


Para adicionar o pacote.

Você pode ver a alteração refletida em nossa lista de pacotes padrão executando:

```julia
] st
```


Para mais informações sobre o modo de pacote, consulte a aula sobre [Ferramentas e Editores](https://julia.quantecon.org/more_julia/tools_editors.html).

### Usando o Gerenciador de Pacotes

Agora, em qualquer terminal Julia no futuro, podemos executar:

```text
using ExamplePackage
```


Para usar suas funções exportadas.

Também podemos encontrar o caminho para isso executando:

```text
using ExamplePackage
pathof(ExamplePackage) # returns path to src/ExamplePackage.jl
```


## Estrutura do Projeto

Vamos descompactar a estrutura do projeto gerado:

- O primeiro diretório `.git`, contém as informações de controle de versão.
-  O diretório `src` contém o código-fonte do projeto - ele deve conter apenas um arquivo (`ExamplePackage.jl`), que lê:

```text
module ExamplePackage

greet() = print("Hello World!")

end # module
```


- Da mesma forma, o diretório `test` deve conter apenas um arquivo (`runtests.jl`), que lê:

```text
using ExamplePackage
using Test

@testset "ExamplePackage.jl" begin
    # Write your own tests here.
end
```




Em particular, o fluxo de trabalho é exportar objetos que queremos testar(`using ExamplePackage`), e testar eles usando o módulo `Test` do Julia.

Os outros arquivos de texto importantes por enquanto são:

- `Project.toml` e `Manifest.toml`, que contêm informações de dependência.  


Em particular, o `Project.toml` contém uma lista de dependências, e o `Manifest.toml` especifica suas versões e subdependências exatas.

- O arquivo `.gitignore` (que pode ser exibido como um arquivo sem título), que contém arquivos e caminhos para o `git` ignorar.  

## Fluxo de Trabalho do Projeto

### Gerenciamento de Dependências

#### Ambientes

Como [antes](https://julia.quantecon.org/more_julia/tools_editors.html#jl-packages), o arquivo .toml define um *ambiente* para o nosso projeto, um conjunto de arquivos que representam as informações de dependência.

Os arquivos reais são gravados na [Linguagem TOML](https://github.com/toml-lang/toml), que é um formato leve para especificar opções de configuração.

Esta informação é o nome de cada pacote em que dependemos, juntamente com as versões exatas desses pacotes.

Essas informações (na prática, o resultado das operações do pacote que executamos) serão refletidas no `ExamplePackage.jl` do diretório do TOML, assim que o ambiente for ativado (selecionado).

Isso permite compartilhar nosso projeto com outros, que podem reproduzir exatamente o estado usado para construí-lo e testá-lo.

Veja o [Pkg3 docs](https://docs.julialang.org/en/v1/stdlib/Pkg/) para mais informações.

#### Pkg Operações

Por enquanto, vamos apenas tentar adicionar uma dependência:

- Ative o ambiente do pacote (a ser executado a partir da base `v1.1`).

```julia
] activate ExamplePackage
```



<dl style='margin: 20px 0;'>
<dt>Isso diz ao Julia para gravar os resultados das operações do pacote no TOML do `ExampleProject`,</dt>
<dd>
e usar as versões dos pacotes especificadas lá.

</dd>

</dl>

Observe que o ambiente base não é especial, exceto que é carregado por um notebook REPL ou Jupyter recém-iniciado.

- Add a package  

```julia
] add Expectations
```


Podemos acompanhar as alterações no TOML, como antes.

Aqui está o  `Manifest.toml`.

![6](https://github.com/pluiz30/Julia-Hub/assets/60633407/4dfb5283-db71-497c-964d-abc215a8af29)
  
Podemos também executar outras operações, como `] up`, `] precompile`, etc.

O pacote de operações está listado em detalhes na aula de [ferramentas e editores](https://julia.quantecon.org/more_julia/tools_editors.html).

Lembre-se de que, para sair do ambiente ativo e retornar à base `(v1.1)`, basta executar:

```julia
] activate
```


### Escrevendo o Código

A ideia básica é trabalhar em `tests/runtests.jl`, enquanto as funções reproduzíveis devem entrar no `src/ExamplePackage.jl`.

Por exemplo, digamos que adicionamos  `Distributions.jl`.

```julia
] activate ExamplePackage
```


```julia
] add Distributions
```


e editamos a fonte (cole-a no próprio arquivo) para ler da seguinte maneira:

```julia
module ExamplePackage

greet() = print("Hello World!")

using Expectations, Distributions

function foo(μ = 1., σ = 2.)
    d = Normal(μ, σ)
    E = expectation(d)
    return E(x -> sin(x))
end

export foo

end # module
```


Vamos tentar chamando isso:

```julia
] activate
```


```julia
using ExamplePackage
ExamplePackage.greet()
```


```julia
foo() # exportado, portanto, não é necessário qualificar o espaço para nome
```


**Nota:** Se você não seguiu as instruções para adicionar um [arquivo de inicialização](https://julia.quantecon.org/more_julia/tools_editors.html#jl-startup-file), pode ser necessário sair do REPL e carregar o pacote novamente.

### Fluxo de Trabalho do Jupyter

Também podemos trabalhar com o pacote em um notebook Jupyter.

Vamos criar um novo diretório de saída em nosso projeto e executar a `jupyter lab` a partir dele.

Crie um novo notebook `output.ipynb`.

![7](https://github.com/pluiz30/Julia-Hub/assets/60633407/dfb1cb07-3ba9-4dbd-a155-370e3c36056e)
  
A partir daqui, podemos usar as funções do nosso pacote como funções de outros pacotes.

Isso nos permite produzir documentos de saída organizados, sem colar toda a base de código.



Também podemos executar operações de pacotes dentro do notebook.

![8](https://github.com/pluiz30/Julia-Hub/assets/60633407/244deb84-7bfb-4801-aa73-0945ea1d1572)
  
A alteração será refletida no arquivo `Project.toml`. 

Observe que, como sempre, tivemos que ativar `ExamplePackage` primeiro antes de fazer nossas alterações de dependência.

```julia
name = "ExamplePackage"
uuid = "f85830d0-e1f0-11e8-2fad-8762162ab251"
authors = ["QuantEcon User <quanteconuser@gmail.com>"]
version = "0.1.0"

[deps]
Distributions = "31c24e10-a181-5473-b8eb-7969acd0382f"
Expectations = "2fe49d83-0758-5602-8f54-1f90ad0d522b"
Parameters = "d96e819e-fc66-5662-9728-84c9c7592b0a"

[extras]
Test = "8dfed614-e22c-5e08-85e1-65c5234f0b40"

[targets]
test = ["Test"]
```


Também haverá mudanças no manifesto.

Certifique-se de adicionar `output/.ipynb_checkpoints` ao seu arquivo `.gitignore`, para que não seja feito o check-in.
Certifique-se de ter ativado o ambiente do projeto  (`] activate ExamplePackage`) antes de tentar propagar as alterações.

### Trabalho Colaborativo

Para alguém obter o pacote, eles simplesmente precisam:

- Executar o seguinte comando

```julia
] dev https://github.com/quanteconuser/ExamplePackage.jl.git
```


Isso colocará o repositório dentro de sua pasta `~/.julia/dev`. 

- Arraste e solte a pasta na área de trabalho do GitHub da maneira usual.


Lembre-se de que o caminho para sua pasta `~/.julia` é:

In [5]:
DEPOT_PATH[1]

"C:\\Users\\cliente\\.julia"

Eles podem então colaborar como fariam em outros repositórios git.

Em particular, eles podem executar:

```julia
] activate ExamplePackage
```


para carregar a lista de dependências e,

```julia
] instantiate
```


para garantir que eles estejam instalados na máquina local.

## Teste de Unidade

É importante garantir que seu código seja bem testado.

Existem alguns tipos diferentes de teste, cada um com finalidades diferentes:

- *Teste de unidade* garante que partes individuais de um projeto funcionem conforme o esperado. 
- *Teste de integração* garante que eles se encaixem conforme o esperado.
- *Teste de regressão* garante que o comportamento permaneça inalterado ao longo do tempo.


Nesta aula, focaremos no teste de unidade:

Em geral, testes de unidade bem escritos (que também protegem contra a regressão, por exemplo, comparando a saída da função com valores codificados) são suficientes para a maioria dos projetos pequenos.

### O Módulo `Test`

Julia fornece recursos de teste por meio de um pacote interno chamado `Test`, pelo qual passamos `using Test`.

O objeto básico é a macro `@test`.

In [2]:
using Test
@test 1 == 1
@test 1 ≈ 1

Test Passed

Os testes serão aprovados se a condição for `true`, ou `fail` caso contrário.

Se um teste está falhando, devemos sinalizá-lo `@test_broken` como abaixo:

In [3]:
@test_broken 1 == 2

Test Broken
  Expression: 1 == 2

Dessa forma, ainda temos acesso a informações sobre o teste, em vez de excluí-lo ou comentá-lo.

Existem outras macros de teste, que verificam coisas como tratamento de erros e estabilidade de tipo.

Usuários avançados podem verificar os [documentos do Julia](https://docs.julialang.org/en/v1/stdlib/Test/).

### Exemplo

Vamos adicionar alguns testes de unidade para a função `foo()` que definimos anteriormente.

Nosso arquivo `tests/runtests.jl` deve parecer com isso.

**Como antes, isso deve ser colado diretamente no arquivo**.

```julia
using ExamplePackage
using Test

@test foo() ≈ 0.11388071406436832
@test foo(1, 1.5) ≈ 0.2731856314283442
@test_broken foo(1, 0) # diz que isso está quebrado
```


E execute-o digitando `] test` em um REPL ativado (ou seja, um REPL no qual você executou `] activate ExamplePackage`).

### Conjuntos de teste

Por padrão, a pasta `runtests.jl` começa com um `@testset`.

Isso é útil para organizar diferentes lotes de testes, mas por enquanto podemos simplesmente ignorá-lo.

Para saber mais sobre conjuntos de testes, consulte [os documentos](https://docs.julialang.org/en/v1/stdlib/Test/index.html#Working-with-Test-Sets-1/).

### Executando os Testes

Existem algumas maneiras diferentes de executar os testes para o seu pacote:

- Execute o real `runtests.jl`, digamos precionando `shift-enter` no Atom.
- Em um novo (`v1.1`) REPL, execute `] test ExamplePackage`.  
- Em um REPL ativado (`ExamplePackage`), basta executar `] test` (lembre-se de que você pode ativar `] activate ExamplePackage`).  

## Integração Contínua com Travis

### Configuração

Por padrão, Travis deve ter acesso a todos os seus repositórios e implantar automaticamente.

Isso inclui repositórios particulares se você estiver em um pacote de desenvolvedores para estudantes ou em um plano acadêmico (o Travis detecta isso automaticamente).

Para mudar isso, vá para "configurações" no seu perfil do GitHub.

![9](https://github.com/pluiz30/Julia-Hub/assets/60633407/cf196ebc-18f3-43b7-8e14-e0c79bd1b998)
  
Clique em “Aplicativos”, depois em “Travis CI”, depois em “Configurar” e escolha os repositórios que deseja rastrear.

### Opções de Compilação

Por padrão, o Travis irá compilar e testar seu projeto (ou seja, “construí-lo”) para novos commits e PRs para cada repo rastreado com um arquivo`.travis.yml`. 

Podemos ver o nosso abrindo-o no Atom:

```julia
# Documentation: http://docs.travis-ci.com/user/languages/julia/
language: julia
os:
- linux
- osx
julia:
- 1.1
- nightly
matrix:
allow_failures:
    - julia: nightly
fast_finish: true
notifications:
email: false
after_success:
- julia -e 'using Pkg; Pkg.add("Coverage"); using Coverage; Codecov.submit(process_folder())'
```


Isso está dizendo a Travis para criar o projeto em Julia, no OSX e Linux, usando Julia v1.1 e a mais recente versão de desenvolvimento.

Ele também diz que, se a versão noturna não funcionar, isso não deve ser registrado como uma falha.

**Nota**: Você não precisará do OSX, a menos que esteja criando algo específico do Mac, como iOS ou Swift.

Você pode excluir essas linhas para acelerar a compilação, da mesma forma para a versão noturna de Julia.

### Trabalhando com Compilação

Como acima, as compilações são acionadas sempre que enviamos alterações ou abrimos uma solicitação de recebimento.

Por exemplo, se enviarmos as alterações para o servidor e clicar no selo Travis (aquele que diz "build") no README, veremos algo como:

![10](https://github.com/pluiz30/Julia-Hub/assets/60633407/555ca7e5-1cb9-481a-b8ae-21967e0b00be)
  
Observe que você pode precisar esperar um pouco e/ou atualizar o navegador.

Isso nos dá uma visão geral de todas as compilações em execução para esse commit.


Para inspecionar uma compilação mais de perto (digamos, se ela falhar), podemos clicar nela e expandir as opções de log:

![11](https://github.com/pluiz30/Julia-Hub/assets/60633407/9970c91c-12d9-478f-91ab-f68c44cdd1e0)
  
Observe que os tempos de compilação aqui não são informativos, porque geralmente não podemos controlar o hardware ao qual nosso trabalho está alocado.



Também podemos cancelar trabalhos específicos, em suas páginas específicas ou clicando no botão cinza "x" no painel.

Por fim, podemos acionar compilações manualmente (sem um novo commit ou PR) na visão geral do Travis.

![12](https://github.com/pluiz30/Julia-Hub/assets/60633407/e222c9e7-a271-4947-aef8-12eaf5575b82)
Para confirmar sem disparar uma compilação, basta adicionar “[ci skip]” em algum lugar dentro da mensagem de confirmação.

### Travis e Solicitação de Requerimento

Um dos principais recursos do Travis é a capacidade de ver rapidamente se os PRs passam nos testes antes de mesclá-los.

Isso acontece automaticamente quando o Travis é ativado em um repositório.

Para um exemplo desse recurso, consulte [este PR](https://github.com/QuantEcon/Games.jl/pull/65/) no repositório `Games.jl`.

## Código de Cobertura

Além do sucesso ou fracasso do nosso conjunto de testes, também queremos saber quanto do nosso código os testes cobrem.

A ferramenta que usamos para fazer isso é chamada [Codecov](http://codecov.io).

### Configuração

Você verá que o Codecov é ativado automaticamente para repositórios públicos com Travis.

Para particulares, você precisará primeiro obter um token de acesso.

Adicione o escopo privado no site do Codecov, assim como fizemos para Travis.

Navegue até a página de configurações de reporte (ou seja, `https://codecov.io/gh/quanteconuser/ExamplePackage.jl/settings` para o nosso repositório) e copie o token.

Em seguida, vá para as configurações do Travis e adicione uma variável de ambiente como abaixo:

![13](https://github.com/pluiz30/Julia-Hub/assets/60633407/5999df4b-5f44-4ecf-89a0-ec9b02d9665c)

### Interpretando Resultados

Clique no selo Codecov para ver a página de compilação do seu projeto.

Isso nos mostra que nossos testes cobrem 50% de nossas funções no `src//`.

**Nota:** Para obter uma visão mais detalhada, podemos clicar no `src//` e no nome do arquivo resultante.

**Nota:** Codecov pode demorar alguns minutos para ser executado pela primeira vez.

![14](https://github.com/pluiz30/Julia-Hub/assets/60633407/0bd82531-fd6d-4531-99c6-bf81eadc106e)  
Isso nos mostra precisamente quais métodos (e partes de métodos) não são testados.

 ## Pull Requests para Projetos Externos do Julia

Como mencionado no [controle de versão](https://julia.quantecon.org/more_julia/version_control.html),  às vezes queremos trabalhar em repositórios externos que também são projetos de Julia.

`] dev` a URL do git (ou nome do pacote, se o projeto for um pacote Julia registrado), que clonará o repositório git para `~/.julia/dev` e sincronizará com o gerenciador de pacotes Julia. 


Por Exemplo, execute:

```julia
] dev Expectations
```


Clonará o repositório `https://github.com/quantecon/Expectations.jl` para `~/.julia/dev/Expectations`.

Certifique-se de fazer isso no ambiente Julia básico (ou seja, depois de executar `] activate` sem argumentos).

Como lembrete, você pode encontrar o local da sua pasta `~/.julia` (chamada de “user depot”), executando:

In [6]:
DEPOT_PATH[1]

"C:\\Users\\cliente\\.julia"

O comando `] dev` também adicionará o destino ao gerenciador de pacotes, para que, sempre que executemos `using Expectations`, Julia carregará nossa cópia clonada desse local.

```julia
using Expectations
pathof(Expectations) #  aponta para o nosso clone git
```


- Arraste essa pasta para o GitHub Desktop. 
- O próximo passo é transferir o pacote original (externo) do site (por exemplo, `https://github.com/quantecon/Expectations.jl`) para a sua conta (`https://github.com/quanteconuser/Expectations.jl` no nosso caso).
  
![15](https://github.com/pluiz30/Julia-Hub/assets/60633407/f60f813d-5143-4577-9c12-e098fc1a3bb5) 

 Edite as configurações no GitHub Desktop (no menu suspenso "Repositório") para refletir o novo URL. 
  
![16](https://github.com/pluiz30/Julia-Hub/assets/60633407/c49d9cdd-85b6-44ba-a08d-6f87e0722b5a)    
  Aqui, alteraríamos o texto destacado para leitura `quanteconuser`, ou qualquer que seja o nosso ID do GitHub.
  
- Se você fizer algumas alterações em um editor de texto e retornar ao GitHub Desktop, verá algo parecido.  



**Nota:** Como antes, estamos editando os arquivos diretamente `~/.julia/dev`, em vez de clonar o repositório novamente.

![17](https://github.com/pluiz30/Julia-Hub/assets/60633407/e1ecd33a-315b-44f9-8b2d-ca03a35a94fb)
  
Aqui, por exemplo, estamos revisando o README.

- Clicando em "commit to master" (lembre-se de que as caixas de seleção ao lado de cada arquivo indicam se deve ser confirmado) e depois pressionar (por exemplo, pressionar "push" no menu suspenso "Repositório") adicionará as alterações confirmadas à sua conta.


Para confirmar isso, podemos verificar o histórico em nossa conta [aqui](https://github.com/quanteconuser/Expectations.jl/commits/master); para saber mais sobre como trabalhar com repositórios git, consulte a aula de [controle de versão](https://julia.quantecon.org/more_julia/version_control.html).

![18](https://github.com/pluiz30/Julia-Hub/assets/60633407/d06c84d6-76c0-4173-9812-231a5dccfe05)
  
A marca de seleção verde indica que os testes do Travis foram aprovados para esse commit.

- Clicando em  “new pull request” na guia de pull requests nos mostrará um instantâneo das alterações e criaremos uma solicitação de recebimento para os mantenedores do projeto revisarem e aprovarem.




![19](https://github.com/pluiz30/Julia-Hub/assets/60633407/8828fb59-0c92-4301-9260-dbc2d1776350)
  
Para saber mais sobre PRs, consulte a seção relevante da aula de [controle de versão](https://julia.quantecon.org/more_julia/version_control.html).

Para obter mais informações sobre *Forking*, consulte os documentos no [GitHub Desktop](https://help.github.com/desktop/guides/contributing-to-projects/cloning-a-repository-from-github-to-github-desktop/) e  [no site do GitHub](https://guides.github.com/activities/forking/)

### Caso com Acesso de Gravação

Se você tiver acesso de gravação ao repositório, podemos pular as etapas anteriores sobre *Forking* e alteração do URL.

Você pode usar `] dev` em um nome de pacote ou na URL do pacote.

```julia
] dev Expectations
```


ou `] dev https://github.com/quanteconuser/Expectations.jl.git` como um exemplo para um pacote não lançado por URL.

O qual clonará novamente o repositório `~/.julia/dev`, e o usará como um pacote Julia.

```julia
using Expectations
pathof(Expectations) # aponta para o nosso clone git
```


Em seguida, arraste essa pasta para o GitHub Desktop como antes.

Então, para trabalhar com o pacote localmente, tudo o que precisamos fazer é abrir o `~/.julia/dev/Expectations` em um editor de texto (como Atom)


![20](https://github.com/pluiz30/Julia-Hub/assets/60633407/ff4856fe-2c83-4305-a646-1447178bbfbf)
  
A partir daqui, podemos editar este pacote exatamente como o criamos e usar o GitHub Desktop para rastrear versões de nossos arquivos de pacote (digamos, depois `] up`, ou editando nosso código fonte, `] add Package`, etc.)

### Removendo um Pacote do Julia

Para “un-dev” um pacote do Julia (por exemplo, se quisermos usar nosso antigo `Expectations.jl`), você pode simplesmente executar:

```julia
] free Expectations
```


Para deletar inteiramente, execute:

```julia
] rm Expectations
```


Em um REPL em que esse pacote está no ambiente ativo.

## Benchmarking

Outro objetivo do teste é garantir que o código não diminua significativamente de uma versão para a outra.

Podemos fazer isso usando as ferramentas fornecidas pelo pacote `BenchmarkTools.jl`.

Veja a aula [necessidade de velocidade](https://julia.quantecon.org/more_julia/need_for_speed.html) para mais detalhes.

## Notas Adicionais 

- A organização [JuliaCI](https://github.com/JuliaCI/) fornece mais utilidades do Julia para integração contínua e testes.

## Revisão

Revisão do fluxo de trabalho para criação, controle de versão e teste de um novo projeto de ponta a ponta.

1. Crie o diretório local do pacote usando o  `PkgTemplates.jl` . 

1. Adicione esse pacote ao gerenciador de pacotes Julia, abrindo um Julia REPL no `~/.julia/dev/ExamplePackage.jl`, certificando-se de que o ambiente ativo é o padrão `(v1.1)`, e pressionando `] dev .` 

1. Arraste e solte essa pasta no GitHub Desktop.

1. Crie um repositório vazio com o mesmo nome no servidor GitHub. 

1. Envie do GitHub Desktop para o servidor. 

1. Abra **a pasta original do projeto** (por exemplo `~/.julia/dev/ExamplePackage.jl`) no Atom.  

1. Faça alterações, teste, itere, etc. Como regra geral, funções como devem estar no diretório `src/` assim que estiverem estáveis e você deve exportá-las desse arquivo com `export func1, func2`. Isto irá exportar todos os métodos de `func1`, `func2`, etc.  

1. Confirme-os no GitHub Desktop à medida que avança (ou seja, você pode e deve usar o controle de versão para rastrear estados intermediários).  

1. Envie para o servidor e veja os resultados do Travis e do Codecov (observe que esses podem levar alguns minutos na primeira vez).  

## Exercícios


### Exercício 1

Seguindo as [instruções para um novo projeto](#project-setup), crie um novo pacote na sua conta do github chamado `NewtonsMethod.jl`.

Neste pacote, você deve criar um pacote simples para executar o Método de Newton usando o código que você criou no exercício 
[metódo de Newton](https://julia.quantecon.org/getting_started_julia/julia_by_example.html#jbe-ex8a) em [Exemplos Introdutórios](https://julia.quantecon.org/getting_started_julia/julia_by_example.html).

Em particular, dentro do seu pacote, você deve ter duas funções:

- `newtonroot(f, f′; x₀, tol = 1E-7, maxiter = 1000)`  
- `newtonroot(f; x₀, tol = 1E-7, maxiter = 1000)`  


Onde a segunda função usa Diferenciação Automática para chamar a primeira.

O pacote deve incluir:

- implementações dessas funções no diretório `/src`.
- conjunto abrangente de testes.
- projetar e manifestar arquivos para replicar seu ambiente de desenvolvimento.
- execução automatizada dos testes com o Travis CI no GitHub.


Para os testes, você deve ter no mínimo:

- uma maneira de lidar com a não convergência (por exemplo, retornar `nothing` como discutido em [tratamento de erros](https://julia.quantecon.org/getting_started_julia/fundamental_types.html#error-handling).  
- vários `@test` para a raiz de uma função conhecida, dado `f` e as derivadas  analíticas`f'`.
- testes dessas raízes usando a versão de diferenciação automática da função.
- teste de encontrar essas raízes com um `BigFloat` e não apenas um `Float64`.  
- teste de não convergência para uma função sem raiz (por exemplo,  $ f(x) = 2 + x^2 $ ).  
- teste para garantir que o `maxiter` está funcionando (por exemplo, o que acontece se você chamar `maxiter = 5`. 
- teste para garantir que `tol` está funcionando. 


E qualquer outra coisa que você possa pensar. Você deve poder executar `] test` do projeto para verificar se o conjunto de testes está em execução e garantir que ele esteja sendo executado automaticamente no Travis CI.

Envie uma confirmação ao repositório que interrompe um dos testes e veja o que o CI do Travis relata após executar a compilação.

### Exercício 2

Assista ao vídeo do youtube [Developing Julia Packages](https://www.youtube.com/watch?v=QVmU29rCjaA) do Chris Rackauckas. A demonstração passa por muitos dos mesmos conceitos desta aula, mas com mais experiência no desenvolvimento orientado a testes e fornecendo mais detalhes para projetos de código aberto.